In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Dataframe for Evergreen Database
Evgn = pd.read_csv("EvergreenDatasetRaw_Dataset_2005-2018_v02.csv")
#Dataframe for individual drugs from Orange Book
Product = pd.read_csv('products.txt',delimiter="~")
#Dataframe for individual patents from Orange Book
Patent = pd.read_csv("patent.txt",delimiter="~")
#Dataframe for linking NDA (New Drug Application Number) to NDC (National Drug Code)
Link = pd.read_csv("20220906_product.csv")
#Dataframe for Historical VA Pricing. Please download the csv from the following link and change the dir you need.
#https://drive.google.com/uc?id=1ZWlyvtMNjOrOtFlFotvnPDoKeaBD0voe
Pricing = pd.read_csv("C:\\Users\\sulli\\Documents\\Code\\Studio 4\\Drug Patents\\historicvapricing.csv")
#Dataframe for CPI. Needed to adjust historical pricing for inflation.


C:\Users\sulli\AppData\Local\Temp\ipykernel_15280\1603298965.py:11: DtypeWarning: Columns (6,13) have mixed types. Specify dtype option on import or set low_memory=False.
  Pricing = pd.read_csv("C:\\Users\\sulli\\Documents\\Code\\Studio 4\\Drug Patents\\historicvapricing.csv")


In [3]:
Product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42569 entries, 0 to 42568
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Ingredient           42569 non-null  object
 1   DF;Route             42569 non-null  object
 2   Trade_Name           42569 non-null  object
 3   Applicant            42569 non-null  object
 4   Strength             42511 non-null  object
 5   Appl_Type            42569 non-null  object
 6   Appl_No              42569 non-null  int64 
 7   Product_No           42569 non-null  int64 
 8   TE_Code              19320 non-null  object
 9   Approval_Date        42569 non-null  object
 10  RLD                  42569 non-null  object
 11  RS                   42569 non-null  object
 12  Type                 42569 non-null  object
 13  Applicant_Full_Name  42569 non-null  object
dtypes: int64(2), object(12)
memory usage: 4.5+ MB


In [4]:
Link.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114624 entries, 0 to 114623
Data columns (total 20 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   productid                         114624 non-null  object 
 1   productndc                        114624 non-null  object 
 2   producttypename                   114624 non-null  object 
 3   proprietaryname                   114615 non-null  object 
 4   proprietarynamesuffix             11158 non-null   object 
 5   nonproprietaryname                114620 non-null  object 
 6   dosageformname                    114624 non-null  object 
 7   routename                         112525 non-null  object 
 8   startmarketingdate                114624 non-null  int64  
 9   endmarketingdate                  4048 non-null    float64
 10  marketingcategoryname             114624 non-null  object 
 11  applicationnumber                 99193 non-null   o

Need to use the Appl_No of the product to link to the NDC number. 

In [5]:
Link = Link[['applicationnumber','productndc']]
Link = Link[~(Link['applicationnumber'].isna())]
Link = Link[Link['applicationnumber'].str.contains('NDA')]
Link['applicationnumber'] = Link['applicationnumber'].apply(lambda x: (x.replace('ANDA','')))
Link['applicationnumber'] = Link['applicationnumber'].apply(lambda x: int(x.replace('NDA','')))
Link = Link[Link['applicationnumber'].isin(Evgn['NDA #'])]

In [6]:
Pricing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7842841 entries, 0 to 7842840
Data columns (total 18 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   Unnamed: 0          int64 
 1   ContractNumber      object
 2   VendorName          object
 3   ContractStartDate   object
 4   ContractStopDate    object
 5   NDCWithDashes       object
 6   SubItemIdentifier   object
 7   PackageDescription  object
 8   Generic             object
 9   TradeName           object
 10  VAClass             object
 11  Covered             object
 12  PrimeVendor         object
 13  Price               object
 14  PriceStartDate      object
 15  PriceStopDate       object
 16  PriceType           object
 17  DateKey             int64 
dtypes: int64(2), object(16)
memory usage: 1.1+ GB


In [7]:
Pricing['NDCWithDashes']

0          00067-3999-42
1          00067-6114-46
2          00067-6171-85
3          68669-0135-05
4          68669-0135-05
               ...      
7842836    60429-0968-10
7842837    60429-0969-01
7842838    60429-0969-10
7842839    60429-0970-01
7842840    60429-0970-10
Name: NDCWithDashes, Length: 7842841, dtype: object

In [8]:
Link

,applicationnumber,productndc
1,202008,0002-1200
19,21411,0002-3227
20,21411,0002-3228
21,21411,0002-3229
22,21520,0002-3230
...,...,...
114618,50808,99207-465
114619,50808,99207-466
114620,50808,99207-467
114621,21758,99207-525


In [9]:
Pricing[['NDC1','NDC2','NDC3']] =  Pricing.NDCWithDashes.str.split('-',expand=True)
Pricing=Pricing.drop(columns=['NDC3'])
Pricing['CleanNDC'] = Pricing['NDC1'] + '-' + Pricing['NDC2']

In [10]:
Pricing = Pricing[Pricing['CleanNDC'].isin(Link['productndc'])]

In [12]:
len(Pricing)

119672

In [13]:
Evgn

,Active Ingredient,NDA #,Product Name,Company,Approval Date,P or E,Date Added,Patent Number,Expiration Date,Codes,Strengths,Delist Request,Orig,Analysis,Added strength,# added strengths,Applied to UC,2nd add,Comments
0,Abacavir Sulfate,20977,Ziagen*,VIIV HLTHCARE,12/17/98,P,pre-2005,5034394,12/18/11,NaN,1,NaN,NaN,Pre-2005,NaN,NaN,NaN,NaN,10/26/72
1,Abacavir Sulfate,20977,Ziagen*,VIIV HLTHCARE,NaN,P,pre-2005,5089500,6/26/09,U-248,1,NaN,NaN,Pre-2005,NaN,NaN,NaN,NaN,NaN
2,Abacavir Sulfate,20977,Ziagen*,VIIV HLTHCARE,NaN,P,pre-2005,6294540,5/14/18,U-65,1,NaN,NaN,Pre-2005,NaN,NaN,NaN,NaN,NaN
3,Abacavir Sulfate,20977,Ziagen*,VIIV HLTHCARE,NaN,P,pre-2005,5034394*PED,6/18/12,NaN,1,NaN,NaN,Pre-2005,NaN,NaN,NaN,NaN,12/22/15
4,Abacavir Sulfate,20977,Ziagen*,VIIV HLTHCARE,NaN,P,pre-2005,5089500*PED,12/26/09,NaN,1,NaN,NaN,Pre-2005,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21287,Zolpidem Tartrate,22328,Intermezzo,PURDUE PHARMA,NaN,P,11/28/11,7682628,2/16/25,U-1194,"001, 002",NaN,Yes,P:UC,NaN,NaN,NaN,NaN,NaN
21288,Zolpidem Tartrate,22328,Intermezzo,PURDUE PHARMA,NaN,E,11/28/11,NaN,11/23/14,NP,"001, 002",NaN,Yes,NP,NaN,NaN,NaN,NaN,NaN
21289,Zolpidem Tartrate,22328,Intermezzo,PURDUE PHARMA,NaN,P,8/28/12,8242131,8/20/29,U-1266,"001, 002",NaN,No,P:UCnew,NaN,NaN,NaN,NaN,NaN
21290,Zolpidem Tartrate,22328,Intermezzo,PURDUE PHARMA,NaN,P,8/28/12,8252809,2/16/25,DP,"001, 002",NaN,No,P:DP,NaN,NaN,NaN,NaN,NaN
